![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/training/english/dl-ner/ner_xlnet.ipynb)

## 0. Colab Setup

In [1]:
import os

# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed spark-nlp

openjdk version "1.8.0_252"
OpenJDK Runtime Environment (build 1.8.0_252-8u252-b09-1~18.04-b09)
OpenJDK 64-Bit Server VM (build 25.252-b09, mixed mode)
     |████████████████████████████████| 215.7MB 61kB/s 
     |████████████████████████████████| 204kB 47.7MB/s 
     |████████████████████████████████| 122kB 5.0MB/s 


# How to train a NER classifier with Xlnet embeddings based on Char CNNs - BiLSTM - CRF

## Download the file into the local File System 
### It is a standard conll2003 format training file

In [2]:
# Download CoNLL 2003 Dataset
import os
from pathlib import Path
import urllib.request


download_path = "./eng.train"


if not Path(download_path).is_file():
    print("File Not found will downloading it!")
    url = "https://github.com/patverga/torch-ner-nlp-from-scratch/raw/master/data/conll2003/eng.train"
    urllib.request.urlretrieve(url, download_path)
else:
    print("File already present.")
    


File Not found will downloading it!


# Read CoNLL Dataset into Spark dataframe and automagically generate features for futures tasks
The readDataset method of the CoNLL class handily adds all the features required in the next steps

In [3]:
import sparknlp
from sparknlp.training import CoNLL

spark = sparknlp.start()
training_data = CoNLL().readDataset(spark, './eng.train')
training_data.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|EU rejects German...|[[document, 0, 47...|[[document, 0, 47...|[[token, 0, 1, EU...|[[pos, 0, 1, NNP,...|[[named_entity, 0...|
|     Peter Blackburn|[[document, 0, 14...|[[document, 0, 14...|[[token, 0, 4, Pe...|[[pos, 0, 4, NNP,...|[[named_entity, 0...|
| BRUSSELS 1996-08-22|[[document, 0, 18...|[[document, 0, 18...|[[token, 0, 7, BR...|[[pos, 0, 7, NNP,...|[[named_entity, 0...|
|The European Comm...|[[document, 0, 18...|[[document, 0, 18...|[[token, 0, 2, Th...|[[pos, 0, 2, DT, ...|[[named_entity, 0...|
|Germany 's repres...|[[document, 0, 21...|[[document, 0, 21...|[[token, 0, 6, Ge...|[[pos, 0, 6, NNP,..

# Define the NER Pipeline 

### This pipeline defines a pretrained Xlnet component and a trainable NerDLApproach which is based on the Char CNN - BiLSTM - CRF

Usually you have to add additional pipeline components before the elmo for the document, sentence and token columns. But Spark NLPs class CoNLL took already care of this for us, awesome!

In [4]:

from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

# Define the pretrained Xlnet model. 
xlnet = XlnetEmbeddings.pretrained().setInputCols("sentence", "token")\
 .setOutputCol("xlnet")\


# Defien the Char CNN - BiLSTM - CRF model. We will feed it the Xlnet tokens 
nerTagger = NerDLApproach()\
  .setInputCols(["sentence", "token", "xlnet"])\
  .setLabelColumn("label")\
  .setOutputCol("ner")\
  .setMaxEpochs(1)\
  .setRandomSeed(0)\Xlnet
  .setVerbose(0)

# put everything into the pipe
pipeline = Pipeline(
    stages = [
      xlnet ,
      nerTagger
  ])

xlnet_base_cased download started this may take some time.
Approximate size to download 415.8 MB
[OK!]


# Fit the Pipeline and get results

In [5]:
ner_df = pipeline.fit(training_data.limit(10)).transform(training_data.limit(10))
ner_df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|               xlnet|                 ner|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|EU rejects German...|[[document, 0, 47...|[[document, 0, 47...|[[token, 0, 1, EU...|[[pos, 0, 1, NNP,...|[[named_entity, 0...|[[word_embeddings...|[[named_entity, 0...|
|     Peter Blackburn|[[document, 0, 14...|[[document, 0, 14...|[[token, 0, 4, Pe...|[[pos, 0, 4, NNP,...|[[named_entity, 0...|[[word_embeddings...|[[named_entity, 0...|
| BRUSSELS 1996-08-22|[[document, 0, 18...|[[document, 0, 18...|[[token, 0, 7, BR...|[[pos, 0, 7, NNP,...|[[named_entity, 0...|[[word_embeddings...|[[

### Checkout only result columns

In [6]:
ner_df.select(*['text', 'ner']).limit(1).show(truncate=False)

+------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                            |ner                                                                                                                                                                                                                                                                                                                                                                                                                                    |
+------------------------------------------------+

## Alternative Albert models 

checkout https://github.com/JohnSnowLabs/spark-nlp-models for alternative models, the following are available :


  * xlnet_base_cased     = https://storage.googleapis.com/xlnet/released_models/cased_L-24_H-1024_A-16.zip    | 24-layer, 1024-hidden, 16-heads
  * xlnet_large_cased    = https://storage.googleapis.com/xlnet/released_models/cased_L-12_H-768_A-12.zip  |  12-layer, 768-hidden, 12-heads. This model is trained on full data (different from the one in the paper).

In [7]:

from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

# Define the pretrained Albert model. 
xlnet_variant = 'xlnet_large_cased'
xlnet = AlbertEmbeddings.pretrained(xlnet_variant).setInputCols("sentence", "token")\
 .setOutputCol("albert")\


# Define the Char CNN - BiLSTM - CRF model. We will Xlnet it the Albert tokens.
nerTagger = NerDLApproach()\
  .setInputCols(["sentence", "token", "xlnet"])\
  .setLabelColumn("label")\
  .setOutputCol("ner")\
  .setMaxEpochs(1)\
  .setRandomSeed(0)\
  .setVerbose(0)

# put everything into the pipe
pipeline = Pipeline(
    stages = [
      xlnet ,
      nerTagger
  ])

ner_df = pipeline.fit(training_data.limit(10)).transform(training_data.limit(50))
ner_df.show()

xlnet_large_cased download started this may take some time.
Approximate size to download 1.3 GB
[OK!]


Py4JJavaError: ignored